In [ ]:
import pandas as pd

# Load file PPM
df_ppm = pd.read_csv("PembacaanPPM.csv", header=None)

# Hitung rerata per titik
rerata_per_titik = df_ppm.mean(axis=1)

# Buat DataFrame H Obs (nT) dengan kolom Titik
df_h_obs = pd.DataFrame({
    'Titik': range(1, len(rerata_per_titik)+1),
    'H Obs (nT)': rerata_per_titik.values
})

# Tampilkan
print(df_h_obs.to_string(index=False))

 Titik  H Obs (nT)
     1    45239.68
     2    45205.18
     3    45249.84
     4    45226.62
     5    45255.20
     6    45164.68
     7    45208.14
     8    45247.28
     9    45219.64
    10    45356.52
    11    45292.58
    12    45257.54
    13    45301.96
    14    45222.02
    15    44982.12
    16    45431.60
    17    45557.84
    18    45528.88
    19    45128.54
    20    45310.02
    21    45365.34
    22    45809.92


In [2]:
# Base (nT) #

import pandas as pd

# Load file csv
df_base = pd.read_csv("timebasesmooth.csv", header=None, names=["Waktu", "Base"])
df_base['Waktu'] = pd.to_datetime(df_base['Waktu'], format="%I:%M:%S %p")
df_base.set_index('Waktu', inplace=True)
df_base = df_base.sort_index()

# Waktu observasi rover
waktu_obs = [
    "7:07:00 AM","7:28:24 AM","7:41:30 AM","8:00:04 AM","8:16:54 AM",
    "8:31:50 AM","8:51:07 AM","9:11:42 AM","9:33:36 AM","10:01:54 AM",
    "10:22:44 AM","10:44:21 AM","11:13:29 AM","11:31:00 AM","12:45:22 PM",
    "12:46:48 PM","1:11:24 PM","1:26:32 PM","1:44:07 PM","1:54:50 PM",
    "2:06:49 PM","2:51:02 PM"
]
df_obs = pd.DataFrame({'Waktu': pd.to_datetime(waktu_obs, format="%I:%M:%S %p")})

#Interpolasi Base (nT)
smoothing_interp = df_base.reindex(df_base.index.union(df_obs['Waktu'])).sort_index()['Base'].interpolate(method='time')

# Ambil hasil untuk waktu observasi rover
df_result = pd.DataFrame({
    'Waktu': df_obs['Waktu'].dt.strftime("%I:%M:%S %p"),
    'Base (nT)': smoothing_interp.loc[df_obs['Waktu']].values
})

print(df_result.to_string(index=False))

      Waktu    Base (nT)
07:07:00 AM 45093.240000
07:28:24 AM 45097.444000
07:41:30 AM 45100.260000
08:00:04 AM 45104.414667
08:16:54 AM 45108.198000
08:31:50 AM 45111.483333
08:51:07 AM 45115.493333
09:11:42 AM 45119.336000
09:33:36 AM 45122.764000
10:01:54 AM 45125.991000
10:22:44 AM 45127.366667
10:44:21 AM 45127.840000
11:13:29 AM 45126.891000
11:31:00 AM 45125.460000
12:45:22 PM 45113.085667
12:46:48 PM 45112.764000
01:11:24 PM 45106.974000
01:26:32 PM 45103.206667
01:44:07 PM 45098.760833
01:54:50 PM 45096.061667
02:06:49 PM 45093.124000
02:51:02 PM 45083.694333


In [3]:
# Variasi Harian (nT)
baseline = 45092.7  # nilai baseline

df_variation = df_result.copy()
df_variation['Variasi Harian (nT)'] = df_variation['Base (nT)'] - baseline


print(df_variation[['Waktu','Variasi Harian (nT)']].to_string(index=False))

      Waktu  Variasi Harian (nT)
07:07:00 AM             0.540000
07:28:24 AM             4.744000
07:41:30 AM             7.560000
08:00:04 AM            11.714667
08:16:54 AM            15.498000
08:31:50 AM            18.783333
08:51:07 AM            22.793333
09:11:42 AM            26.636000
09:33:36 AM            30.064000
10:01:54 AM            33.291000
10:22:44 AM            34.666667
10:44:21 AM            35.140000
11:13:29 AM            34.191000
11:31:00 AM            32.760000
12:45:22 PM            20.385667
12:46:48 PM            20.064000
01:11:24 PM            14.274000
01:26:32 PM            10.506667
01:44:07 PM             6.060833
01:54:50 PM             3.361667
02:06:49 PM             0.424000
02:51:02 PM            -9.005667


In [4]:
# Terkoreksi Variasi Harian #
df_corrected = pd.DataFrame({
    'Waktu': df_result['Waktu'],
    'Terkoreksi Variasi Harian (nT)': df_h_obs['H Obs (nT)'] - df_variation['Variasi Harian (nT)']
})

print(df_corrected.to_string(index=False))

      Waktu  Terkoreksi Variasi Harian (nT)
07:07:00 AM                    45239.140000
07:28:24 AM                    45200.436000
07:41:30 AM                    45242.280000
08:00:04 AM                    45214.905333
08:16:54 AM                    45239.702000
08:31:50 AM                    45145.896667
08:51:07 AM                    45185.346667
09:11:42 AM                    45220.644000
09:33:36 AM                    45189.576000
10:01:54 AM                    45323.229000
10:22:44 AM                    45257.913333
10:44:21 AM                    45222.400000
11:13:29 AM                    45267.769000
11:31:00 AM                    45189.260000
12:45:22 PM                    44961.734333
12:46:48 PM                    45411.536000
01:11:24 PM                    45543.566000
01:26:32 PM                    45518.373333
01:44:07 PM                    45122.479167
01:54:50 PM                    45306.658333
02:06:49 PM                    45364.916000
02:51:02 PM                    4

In [5]:
# Hitung Anomali Magnet Total (nT) #
igrf = 44936

df_total_anomaly = df_corrected.copy()
df_total_anomaly['Anomali Magnet Total (nT)'] = df_total_anomaly['Terkoreksi Variasi Harian (nT)'] - igrf

print(df_total_anomaly[['Anomali Magnet Total (nT)']].to_string(index=False))

 Anomali Magnet Total (nT)
                303.140000
                264.436000
                306.280000
                278.905333
                303.702000
                209.896667
                249.346667
                284.644000
                253.576000
                387.229000
                321.913333
                286.400000
                331.769000
                253.260000
                 25.734333
                475.536000
                607.566000
                582.373333
                186.479167
                370.658333
                428.916000
                882.925667
